# 回転の推定II:異方性誤差

In [ ]:
import sys
from itertools import product

import numpy as np
from scipy.stats import random_correlation, special_ortho_group
from scipy.spatial.transform import Rotation

sys.path.append('../libs')
import util

In [ ]:
A_bar = util.load_point_cloud()
points_num = A_bar.shape[1]
print(points_num)
util.plot_3d(A_bar)

In [ ]:
sigma = np.array([[1., 1.2, 1.4]])
cov = sigma * sigma.T * random_correlation.rvs((1.2, 0.8, 1.0))
util.plot_3d(np.random.multivariate_normal(np.zeros(3), cov, 10000).T)

In [ ]:
sigma = np.array([[1., 1.2, 1.4]])
cov_a = sigma * sigma.T * random_correlation.rvs((0.5, 1.2, 1.3))
noise_level = 3e-3
A = A_bar + noise_level * np.random.multivariate_normal(np.zeros(3), cov_a, points_num).T
util.plot_3d(A)

In [ ]:
ideal_R = special_ortho_group.rvs(3)
print(ideal_R)

In [ ]:
sigma = np.array([[1.2, 0.8, 1.1]])
cov_a_prime = sigma * sigma.T * random_correlation.rvs((0.1, 0.2, 2.7))
A_prime = ideal_R @ A_bar + noise_level * np.random.multivariate_normal(np.zeros(3), cov_a_prime, points_num).T
util.plot_3d(A_prime)

## 特異値分解の場合

In [ ]:
R1 = util.estimate_R_using_SVD(A, A_prime)
print('error:', util.eval_R_error(R1, ideal_R))

## 5.3 四元数表現による回転推定

In [ ]:
xi = np.stack([
    np.stack([
        A_prime[0] - A[0],
        np.zeros(points_num),
        -(A_prime[2] + A[2]),
        A_prime[1] + A[1]
    ]),
    np.stack([
        A_prime[1] - A[1],
        A_prime[2] + A[2],
        np.zeros(points_num),
        -(A_prime[0] + A[0])
    ]),
    np.stack([
        A_prime[2] - A[2],
        -(A_prime[1] + A[1]),
        A_prime[0] + A[0],
        np.zeros(points_num)
    ])
])
print(xi.shape)

In [ ]:
T = np.array([
    [
        [-1,  0,  0,  1,  0,  0],
        [ 0,  0,  0,  0,  0,  0],
        [ 0,  0, -1,  0,  0, -1],
        [ 0,  1,  0,  0,  1,  0]
    ], [
        [ 0, -1,  0,  0,  1,  0],
        [ 0,  0,  1,  0,  0,  1],
        [ 0,  0,  0,  0,  0,  0],
        [-1,  0,  0, -1,  0,  0]
    ], [
        [ 0,  0, -1,  0,  0,  1],
        [ 0, -1,  0,  0, -1,  0],
        [ 1,  0,  0,  1,  0,  0],
        [ 0,  0,  0,  0,  0,  0]
    ]
])
print(T.shape)

In [ ]:
cov_joined = np.block([[cov_a, np.zeros([3, 3])], [np.zeros([3, 3]), cov_a_prime]])
print(cov_joined)

V_0 = np.zeros([3, 3, T.shape[1], T.shape[1]])
for k, l in product(range(3), repeat=2):
    V_0[k, l] = T[k] @ cov_joined @ T[l].T
print(V_0.shape)

## 5.4 FNS法による最適化

In [ ]:
def calc_M(W, xi):
    dim = xi.shape[1]
    M = np.zeros([dim, dim])
    for k, l in product(range(3), repeat=2):
        M += W[k, l] * xi[k] @ xi[l].T
    return M

In [ ]:
def calc_L(W, q, xi, V_0):
    _, dim, points_num = xi.shape
    v = np.zeros([3, points_num])
    for k, l in product(range(3), repeat=2):
        v[k] += W[k, l] * xi[l].T @ q
    L = np.zeros([dim, dim])
    for k, l in product(range(3), repeat=2):
        L += v[k].T @ v[l] * V_0[k, l]
    return L

In [ ]:
def FNS_method(xi, V_0):
    # step 1
    q0 = np.zeros(4)
    W = np.eye(3)
    iters = 1

    while True:
        # step 2
        X = calc_M(W, xi) - calc_L(W, q0, xi, V_0)
        # step 3
        w, v = np.linalg.eigh(X)
        q = v[:, np.argmin(w)]
        # step 4
        if np.allclose(q, q0) or np.allclose(q, -q0):
            return q, iters
        W_inv = np.zeros_like(W)
        for k, l in product(range(3), repeat=2):
            W_inv[k, l] = q @ V_0[k, l] @ q
        W = np.linalg.inv(W_inv)
        q0 = q
        iters += 1

In [ ]:
q, iters = FNS_method(xi, V_0)
R2 = Rotation.from_quat(q[[1, 2, 3, 0]]).as_dcm()
print('iterations:', iters)
print('error:', util.eval_R_error(R2, ideal_R))

In [ ]:
util.plot_3d_multi(R2 @ A, A_prime)

## 5.5 同次拘束条件による解法

In [ ]:
zeros = np.zeros([3, points_num])
xi = np.stack([
    np.concatenate([A, zeros, zeros, -A_prime[[0]]]),
    np.concatenate([zeros, A, zeros, -A_prime[[1]]]),
    np.concatenate([zeros, zeros, A, -A_prime[[2]]])
])
del zeros
print(xi.shape)

In [ ]:
T = np.zeros([3 ,10, 6])
for i in range(3):
    T[i, i * 3, 0] = T[i, i * 3 + 1, 1] = T[i, i * 3 + 2, 2] = 1
    T[i, 9, 3 +  i] = -1
print(T.shape)
print(T)

In [ ]:
V_0 = np.zeros([3, 3, T.shape[1], T.shape[1]])
for k, l in product(range(3), repeat=2):
    V_0[k, l] = T[k] @ cov_joined @ T[l].T
print(V_0.shape)

In [ ]:
def projection_matrix(u):
    orthogonal_basis = np.array([
        [u[1], u[0], 0, u[4], u[3], 0, u[7], u[6], 0, 0],
        [0, u[2], u[1], 0, u[5], u[4], 0, u[8], u[7], 0],
        [u[2], 0, u[0], u[5], 0, u[3], u[8], 0, u[6], 0],
        [2*u[0], 0, 0, 2*u[3], 0, 0, 2*u[6], 0, 0, -2*u[9]],
        [0, 2*u[1], 0, 0, 2*u[4], 0, 0, 2*u[7], 0, -2*u[9]],
        [0, 0, 2*u[2], 0, 0, 2*u[5], 0, 0, 2*u[8], -2*u[9]],
    ]).T

    constraint_num = orthogonal_basis.shape[1]
    # Gram–Schmidt process
    Q, _ = np.linalg.qr(orthogonal_basis)
    P = np.eye(10)
    for i in range(6):
        P -= np.outer(Q[:, i], Q[:, i])
    return P, constraint_num

In [ ]:
def EFNS_method(xi, V_0):
    # step 1
    u = np.array([1., 0., 0.,
                  0., 1., 0.,
                  0., 0., 1., 1.])
    u /= np.linalg.norm(u)
    W = np.eye(3)
    iters = 1

    while True:
        # step 2
        M = calc_M(W, xi)
        L = calc_L(W, u, xi, V_0)
        # step 3, 4
        P, constraint_num = projection_matrix(u)
        # step 5
        X = P @ (M - L) @ P
        # step 6
        w, v = np.linalg.eigh(X)
        v = v[:, np.argsort(w)[:constraint_num + 1]]
        # step 7
        u_hat = np.zeros_like(u)
        for i in range(v.shape[1]):
            u_hat += u @ v[:, i] * v[:, i]
        # step 8
        u_prime = P @ u_hat
        u_prime /= np.linalg.norm(u_prime)
        if np.allclose(u_prime, u) or np.allclose(u_prime, -u):
            return u_prime, iters
        u += u_prime
        u /= np.linalg.norm(u)
        W_inv = np.zeros_like(W)
        for k, l in product(range(3), repeat=2):
            W_inv[k, l] = u @ V_0[k, l] @ u
        W = np.linalg.inv(W_inv)
        iters += 1

In [ ]:
u, iters = EFNS_method(xi, V_0)
R3 = u[:-1].reshape(3, 3) / u[-1]
print('iterations:', iters)
print('error:', util.eval_R_error(R3, ideal_R))

In [ ]:
util.plot_3d_multi(R3 @ A, A_prime)

## 6.6 最尤推定による回転の最適化
（章は違うがやっていることは同じなので混ぜた）

In [ ]:
def calc_W(cov_a, cov_a_prime, R):
    return np.linalg.inv(R @ cov_a @ R.T + cov_a_prime)

In [ ]:
def calc_g(A, A_prime, R, W, cov_a):
    WE = W @ (A_prime - R @ A)
    g = (-np.cross(R @ A, WE, axis=0) + np.cross(WE, R @ cov_a @ R.T @ WE, axis=0)).sum(axis=1)
    return g

In [ ]:
def calc_H(A, R, W):
    tmp = np.stack([
        np.cross(R @ A, W[:, 0], axis=0),
        np.cross(R @ A, W[:, 1], axis=0),
        np.cross(R @ A, W[:, 2], axis=0),
    ], axis=1)
    return np.cross(tmp, R @ A, axisa=1, axisb=0, axisc=1).sum(axis=2)

In [ ]:
def calc_J(A, A_prime, cov_a, cov_a_prime, R):
    W = calc_W(cov_a, cov_a_prime, R)
    E = A_prime - R @ A
    return (E * (W @ E)).sum()

In [ ]:
def skew_matrix(v):
    return np.array([
        [0, -v[2], v[1]],
        [v[2], 0, -v[0]],
        [-v[1], v[0], 0]
    ])

In [ ]:
def lie_optimize(A, A_prime, cov_a, cov_a_prime):
    # step 1
    R = util.estimate_R_using_SVD(A, A_prime)
    c = 0.0001

    while True:
        W = calc_W(cov_a, cov_a_prime, R)
        # step 2
        g = calc_g(A, A_prime, R, W, cov_a)
        H = calc_H(A, R, W)
        while True:
            # step 3
            omega = np.linalg.solve(H + c * np.eye(3), -g)
            # step 4
            t = np.linalg.norm(omega)
            A_omega = skew_matrix(omega / t)
            exp_omega = np.eye(3) + np.sin(t) * A_omega + (1 - np.cos(t)) * A_omega @ A_omega
            new_R = exp_omega @ R
            # step 5
            J = calc_J(A, A_prime, cov_a, cov_a_prime, R)
            new_J = calc_J(A, A_prime, cov_a, cov_a_prime, new_R)
            if new_J <= J:
                break
            c *= 10
        # step 6
        if np.linalg.norm(omega) < 1e-10:
            return new_R
        R = new_R
        c /= 10

In [ ]:
R4 = lie_optimize(A, A_prime, cov_a, cov_a_prime)
print('initial error:', util.eval_R_error(R1, ideal_R))
print('final error:', util.eval_R_error(R4, ideal_R))

In [ ]:
util.plot_3d_multi(R4 @ A, A_prime)